In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [6]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
x_train = mnist.train.images
y_train = mnist.train.labels
x_test = mnist.test.images
y_test = mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
#for NN layers
class layer:
    def __init__(self, inputs, in_size, out_size, activation_function=None):
#         self.W = tf.Variable(tf.zeros([in_size, out_size]))
        self.W = tf.Variable(tf.random_normal([in_size, out_size]))
        self.b = tf.Variable(tf.zeros([1,out_size]))
        self.Wx_plus_b = tf.matmul(inputs, self.W) + self.b
        self.activation_function = activation_function
    def output(self):
        if self.activation_function == None:
            result = self.Wx_plus_b
        else :
            result = self.activation_function(self.Wx_plus_b)
        return result

In [8]:
# for convolution
class convolution:
    def __init__(self, shape):
        self.W = tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1))
        self.b = tf.Variable(tf.constant(0.1, shape=[shape[3]]))
    def conv2d(self, inputs, padding='SAME'):
        return tf.nn.conv2d(inputs, self.W, strides=[1,1,1,1], padding=padding)
    def max_pooling_nxn(self, inputs, n):
        return tf.nn.max_pool(inputs, ksize=[1,n,n,1], strides=[1,n,n,1], padding='SAME')
    def conv_and_pooling(self, inputs, activation_function=None, pooling_size=2):
        if activation_function==None:
            h_conv =self.conv2d(inputs)+self.b
        else:
            h_conv = activation_function(self.conv2d(inputs)+self.b)    
        return self.max_pooling_nxn(h_conv, pooling_size)

In [9]:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

#for dropout
keep_prob = tf.placeholder(tf.float32)

#dealing eith inputs. 
# -1 for not considering input number of images
# 28,28 size per iamge
# 1 for channel. 1 for grayscale ; 3 for color image
x_image=tf.reshape(x,[-1,28,28,1])

In [11]:
#AlexNet => 5 convolution layers & 3 fully connected neural network

conv1 = convolution([3,3,1,24])
conv2 = convolution([3,3,24,48])
conv3 = convolution([3,3,48,144])
# conv4 = convolution([3,3,96,192])
# conv5 = convolution([3,3,192,192])

output_conv1 = conv1.conv_and_pooling(x_image, tf.nn.relu)
output_conv2 = conv2.conv_and_pooling(output_conv1, tf.nn.relu)
output_conv3 = conv3.conv_and_pooling(output_conv2, tf.nn.relu, pooling_size=2)
# output_conv4 = conv4.conv_and_pooling(output_conv3, tf.nn.relu, pooling_size=1)
# output_conv5 = conv5.conv_and_pooling(output_conv4, tf.nn.relu, pooling_size=1)

# h_pool_flat = tf.reshape(output_conv2, [-1,7*7*40])
h_pool_flat = tf.reshape(output_conv3, [-1,4*4*144])

# layer1 = layer(h_pool_flat, 7*7*64, 500, tf.nn.relu)
# layer1 = layer(h_pool_flat, 7*7*64, 10, tf.nn.softmax)
# layer1 = layer(h_pool_flat, 7*7*40, 10, tf.nn.softmax)
layer1 = layer(h_pool_flat, 4*4*144, 100, tf.nn.sigmoid)
# layer2 = layer(layer1.output(), 400, 100, tf.nn.sigmoid)
layer3 = layer(layer1.output(), 100, 10, tf.nn.softmax)
# layer2 = layer(layer1.output(), 500, 100, tf.nn.relu)
# layer3 = layer(layer2.output(), 100, 10, tf.nn.softmax)

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer1.output()))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=layer3.output()))
# train_step = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
# train_step =  tf.train.MomentumOptimizer(0.005 , 0.9).minimize(loss)
train_step = tf.train.AdamOptimizer(0.003).minimize(loss)


correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(layer3.output(), 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

batch_size = 100
batches = x_train.shape[0]//batch_size


In [13]:
for epoch in range(31):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)]})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

start 0
epoch: 0 loss: 1.84368 accuracy: 0.610073
start 1
start 2
epoch: 2 loss: 1.6747 accuracy: 0.784782
start 3
start 4
epoch: 4 loss: 1.59326 accuracy: 0.873637
start 5
start 6
epoch: 6 loss: 1.48915 accuracy: 0.975455
start 7
start 8
epoch: 8 loss: 1.48703 accuracy: 0.977382
start 9
start 10
epoch: 10 loss: 1.4909 accuracy: 0.972709
start 11
start 12
epoch: 12 loss: 1.48575 accuracy: 0.977418
start 13
start 14
epoch: 14 loss: 1.4932 accuracy: 0.969928
start 15
start 16
epoch: 16 loss: 1.48647 accuracy: 0.976691
start 17
start 18
epoch: 18 loss: 1.49037 accuracy: 0.972818
start 19
start 20
epoch: 20 loss: 1.48718 accuracy: 0.976073
start 21
start 22
epoch: 22 loss: 1.48819 accuracy: 0.974637
start 23
start 24
epoch: 24 loss: 1.49065 accuracy: 0.973182
start 25
start 26
epoch: 26 loss: 1.52268 accuracy: 0.939109
start 27
start 28
epoch: 28 loss: 1.48568 accuracy: 0.976546
start 29
start 30
epoch: 30 loss: 1.48685 accuracy: 0.975455
Test accuracy: 0.9781


In [14]:
for epoch in range(51):
    print "start "+str(epoch)
    for batch in range(batches):
        sess.run(train_step, feed_dict={x: x_train[batch_size*batch:batch_size*(batch+1)], y: y_train[batch_size*batch:batch_size*(batch+1)]})
    if epoch%2==0:
        print "epoch: "+str(epoch)+" loss: "+str(sess.run(loss, feed_dict={x: x_train, y: y_train}))+" accuracy: "+str(sess.run(accuracy, feed_dict={x: x_train, y: y_train}))
print "Test accuracy: "+str(sess.run(accuracy, feed_dict={x: x_test, y: y_test}))

start 0
epoch: 0 loss: 1.49242 accuracy: 0.970764
start 1
start 2
epoch: 2 loss: 1.48951 accuracy: 0.973691
start 3
start 4
epoch: 4 loss: 1.49271 accuracy: 0.970219
start 5
start 6
epoch: 6 loss: 1.49432 accuracy: 0.969509
start 7
start 8
epoch: 8 loss: 1.50694 accuracy: 0.956582
start 9
start 10
epoch: 10 loss: 1.49721 accuracy: 0.965691
start 11
start 12
epoch: 12 loss: 1.49082 accuracy: 0.972109
start 13
start 14
epoch: 14 loss: 1.50147 accuracy: 0.961891
start 15
start 16
epoch: 16 loss: 1.50071 accuracy: 0.961709
start 17
start 18
epoch: 18 loss: 1.49137 accuracy: 0.9712
start 19
start 20
epoch: 20 loss: 1.49353 accuracy: 0.969382
start 21
start 22
epoch: 22 loss: 1.50799 accuracy: 0.956346
start 23
start 24
epoch: 24 loss: 1.49539 accuracy: 0.967746
start 25
start 26
epoch: 26 loss: 1.50548 accuracy: 0.958637
start 27
start 28
epoch: 28 loss: 1.49989 accuracy: 0.963309
start 29
start 30
epoch: 30 loss: 1.52904 accuracy: 0.9358
start 31
start 32
epoch: 32 loss: 1.5015 accuracy: 0

In [16]:
sess.close()